In [1]:
import cv2
import numpy as np 
import os
import yaml
from yaml.loader import SafeLoader


In [2]:
#load yaml
with open('data.yaml',mode='r') as f:
    data_yaml=yaml.load(f,Loader=SafeLoader)
    
labels=data_yaml['names']
print(labels)


['person', 'car', 'chair', 'bottle', 'pottedplant', 'bird', 'dog', 'sofa', 'bicycle', 'horse', 'boat', 'motorbike', 'cat', 'tvmonitor', 'cow', 'sheep', 'aeroplane', 'train', 'diningtable', 'bus']


In [3]:
#Load YOLO model
yolo =cv2.dnn.readNetFromONNX('./Model4/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [4]:
#load the test image
img = cv2.imread('/Users/sandeepchaudhary/Desktop/YOLO/2_predictions/45.jpg')
image = img.copy()
#cv2.imshow('image',image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
row,col,d= image.shape
#get the YOLO predictions
#step-1 convert the image into a square image (array)
max_rc=max(row,col)
input_image=np.zeros((max_rc,max_rc,3),dtype=np.uint8)
input_image[0:row,0:col]=image
#step-2 get prediction from the square array
INPUT_WH_YOLO=640
blob =cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
yolo.setInput(blob)
preds=yolo.forward()#predictions from yolo model

In [5]:
print(preds.shape)

(1, 25200, 25)


In [7]:
#Non Maximum suppresion Filter
#filter detections based on confidence score(0.4) and probability score(0.25)
detections=preds[0]
boxes=[]
confidences=[]
classes=[]

#calculate the width and height of the image
image_w,image_h=input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4]# confidence of detection of an object
    if confidence > 0.4:
        class_score = row[5:].max() #maximum probability out of the 20 objects
        class_id = row[5:].argmax() #get the index position at which the max probability occur
        if class_score > 0.25:
            cx,cy,w,h = row[0:4]
            #construct the bounding box from 4 values
            #left,top,width and height 
            left = int((cx-0.5*w)*x_factor)
            top = int((cy-0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            box=np.array([left,top,width,height])

            #append values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

#clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

#NMS
index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()


In [11]:
#draw the bounding boxes
for ind in index:
    #extract bounding boxes
    x,y,w,h = boxes_np[ind]
    bb_conf=confidences[ind]*100
    classes_id=classes[ind]
    class_name=labels[classes_id]
    text=f'{class_name}:{bb_conf}%'
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),(255,255,255),-1)
    cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)   

In [12]:
boxes_np

[[863, 832, 590, 1721],
 [862, 827, 607, 1730],
 [854, 816, 603, 1844],
 [853, 820, 612, 1842],
 [881, 814, 584, 1947],
 [875, 820, 588, 1932],
 [866, 825, 583, 1933],
 [881, 844, 570, 1882]]

In [13]:
cv2.imshow('yolo_predictions',image)
cv2.waitKey(0)
cv2.destroyAllWindows()